# GMI Attack target network
Use KITTI image to attack Target (GTA5) Network
Step 1: Train GAN on KITTI



In [2]:
import torch
from models import  DeepLabV3
import torch.nn as nn

In [3]:
# Models
# Generator
# Discriminator
# Target
Target_model = DeepLabV3().cuda()

Target_model = Target_model.train()
# use hook to get internal input
# print(Target_model)

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
BNLoss = []
def get_BN(name):
    def hook(model, input, output):
        running_mean = model.running_mean
        running_var = model.running_var
        

        nch = input[0].detach().shape[1]
        mean = input[0].detach().mean([0, 2, 3])
        var = input[0].detach().permute(1, 0, 2, 3).contiguous().view([nch, -1]).var(1, unbiased=False)

        BNLoss.append(torch.norm(running_var.data - var, 2) + torch.norm(
            running_mean.data - mean, 2))

    return hook

    
for m in Target_model.backbone.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.register_forward_hook(get_BN(m))
    # if m.startswith('layer'):
    #     Target_model.backbone[m][-1].bn3.register_forward_hook(get_BN(m))




d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [97]:
dummy = torch.rand(2,3,64,128).cuda()
y = Target_model(dummy)['out']
# print(y.size()) #B 3 256 512 -> B 19 32 64 

# for l in activation:
#     print(l,activation[l].size())

print(sum(BNLoss))
# for l in BNLoss:
#     print(BNLoss)


tensor(108.4299, device='cuda:0')
